In [ ]:
from langchain_core.tools import tool
import requests
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import json

In [13]:
# tool create

@tool
def get_conversion_factor(base_currency:str,target_currency:str) -> float:
    """
    This function fetches the currency convestion factor betwen a given base currency and a target currency 
    """
    # Exchange rate api
    url = f'https://v6.exchangerate-api.com/v6/fc09a5a5c74ed811e8bed60a/pair/{base_currency}/{target_currency}'
    # url = "https://v6.exchangerate-api.com/v6/fc09a5a5c74ed811e8bed60a/latest/USD/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()


@tool
def convert(base_currency_value:int,conversion_rate:Annotated[float,InjectedToolArg]) -> float: # this annotted means that  llm do no try to fill this argument i (developer) will inject this value after the running the earlier tool
    """
    given a currency conversion rate this dunction calculetes the target currency value from a given base currency value
    """

    return base_currency_value * conversion_rate

In [14]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})


{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1770249601,
 'time_last_update_utc': 'Thu, 05 Feb 2026 00:00:01 +0000',
 'time_next_update_unix': 1770336001,
 'time_next_update_utc': 'Fri, 06 Feb 2026 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 90.4427}

In [15]:
convert.invoke({'base_currency_value':10,'conversion_rate':83.45})


834.5

In [ ]:
## binding

llm = ChatOpenAI()

In [ ]:
llm_tools = llm.bind_tools([get_conversion_factor,convert])


messages =  [hum]

In [ ]:
messages =  [HumanMessage('what is the conversion factor bectween USD and INT , and based on that can you convert 10 usd to the inr')]

In [ ]:
messages

In [ ]:
ai_message = llm_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

In [ ]:
for tool_call in ai_message.tool_calls:
    print(tool_call)
    # execute the first tool and get the value of the conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        print(tool_message1)

        # fetch this conversion rate
        conversion_rate = json_loads(tool_message1.content)['conversion_rate']
        # append this tool messages in the messaage varialbe
        messages.append(tool_message1)
    # execute the secound tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        # fetch the currenct argument 
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)
        

In [ ]:
llm_tools.invoke(messages)